# Chapter 11 統計的仮説検定

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
%precision 3
np.random.seed(1111)

In [2]:
df = pd.read_csv('../data/ch11_potato.csv')

sample = np.array(df['重さ'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [3]:
s_mean = np.mean(sample)
s_mean

128.4507142857143

## 11.1 |　統計的仮説検定とは

### 11.1.1　統計的仮説検定の基本

In [4]:
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.68118313069039

In [5]:
z = (s_mean -130)/np.sqrt(9/14)
z

-1.932298779026813

In [6]:
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [7]:
# p 値
rv.cdf(z)

0.026661319523126635

### 11.1.2　片側検定と両側検定

In [8]:
z = (s_mean - 130)/np.sqrt(9/14)
z

-1.932298779026813

In [9]:
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

片側検定の方が両側検定より帰無仮説を棄却しやすい

In [10]:
rv.cdf(z) * 2

0.05332263904625327

### 11.1.3　仮説検定における 2つの過誤

__第一種の過誤__  帰無仮説が正しいときに, 帰無仮説を棄却してしまう過誤  
__第二種の過誤__  対立仮説が正しいときに, 帰無仮説を採択してしまう過誤

In [11]:
rv = stats.norm(130, 3)

In [12]:
c = stats.norm().isf(0.95)
n_sample = 10000
cnt = 0

for _ in range(n_sample):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130)/np.sqrt(9/14)
    
    if z < c:
        cnt += 1
        
cnt/n_sample

0.053

In [13]:
rv = stats.norm(128, 3)

In [14]:
c = stats.norm().isf(0.95)
n_sample = 10000
cnt = 0

for _ in range(n_sample):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130)/np.sqrt(9/14)
    
    if z >= c:
        cnt += 1
        
cnt/n_sample

0.197

## 11.2 |　基本的な仮説検定

### 11.2.1　正規分布の母平均の検定 (母分散概知)

In [15]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
         print('Accept the null hypothesis')
    else:
         print('Reject the null hypothesis')
        
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p-value: {p:.3f}')

In [16]:
pmean_test(sample, 130, 9)

Accept the null hypothesis
p-value: 0.053


### 11.2.2 正規分布の母分散の検定

In [17]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('Accept the null hypothesis')
    else:
        print('Reject the null hypothesis')
        
    if y < 0:
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p-value: {p:.3f}')

In [18]:
pvar_test(sample, 9)

Accept the null hypothesis
p-value: 0.085


### 11.2.3　正規分布の母平均の検定 (母分散未知)

In [19]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean -mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('Accept the null hypothesis')
    else:
        print('Reject the null hypothesis')
        
    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p-value: {p:.3f}')

In [20]:
pmean_test(sample, 130)

Accept the null hypothesis
p-value: 0.169


In [21]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.4551960206404198, 0.16933464230414275)

## 11.3 | 2 標本問題に関する仮説検定

### 11.3.1　対応のある t 検定

In [22]:
training_df = pd.read_csv('../data/ch11_training_rel.csv')
print(training_df.shape)
training_df.head()

(20, 2)


,前,後
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


In [23]:
training_df['差'] = training_df['後'] - training_df['前']
training_df.head()

,前,後,差
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


In [24]:
t, p = stats.ttest_1samp(training_df['差'], 0)
p

0.04004419061842953

In [25]:
t, p = stats.ttest_rel(training_df['後'], training_df['前'])
p

0.04004419061842953

### 11.3.2　対応のない t 検定

In [26]:
training_df = pd.read_csv('../data/ch11_training_ind.csv')
print(training_df.shape)
training_df.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [27]:
t , p = stats.ttest_ind(training_df['A'], training_df['B'], equal_var=False)
p

0.08695731107259361

### 11.3.3　ウィルコクソンの符号付き順位検定

In [28]:
training_df = pd.read_csv('../data/ch11_training_rel.csv')
toy_df = training_df[:6].copy()
toy_df

,前,後
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


In [29]:
diff = toy_df['後'] - toy_df['前']
toy_df['差'] = diff
toy_df

,前,後,差
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [30]:
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['順位'] = rank
toy_df

,前,後,差,順位
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [31]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(8, 13)

r_minus の方が小さいので検定統計量になる.

In [32]:
T, p = stats.wilcoxon(training_df['前'], training_df['後'])
p

0.037999792729223686

### 11.3.4　マン・ホイットニーの U 検定

In [33]:
training_df = pd.read_csv('../data/ch11_training_ind.csv')
toy_df = training_df[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [34]:
rank = stats.rankdata(np.concatenate([toy_df['A'],
                                     toy_df['B']]))
rank_df = pd.DataFrame({'A': rank[:5],
                       'B': rank[5:10]}).astype(int)
rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


In [35]:
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

7.0

In [36]:
u, p = stats.mannwhitneyu(training_df['A'], training_df['B'],
                         alternative='two-sided')
p

0.05948611166127324

### 11.3.5　カイ二乗検定

In [37]:
ad_df = pd.read_csv('../data/ch11_ad.csv')
n = len(ad_df)
print(n)
ad_df.head()

1000


,広告,購入
0,B,しなかった
1,B,しなかった
2,A,した
3,A,した
4,B,しなかった


In [38]:
ad_cross = pd.crosstab(ad_df['広告'], ad_df['購入'])
ad_cross

購入,した,しなかった
広告,,
A,49,351
B,51,549


In [39]:
ad_cross['した'] / (ad_cross['した'] + ad_cross['しなかった'])

広告
A    0.1225
B    0.0850
dtype: float64

In [40]:
n_yes, n_not = ad_cross.sum()
n_yes, n_not

(100, 900)

In [41]:
n_adA, n_adB = ad_cross.sum(axis=1)
n_adA, n_adB

(400, 600)

In [42]:
# 期待度数を計算
ad_ef = pd.DataFrame({'した': [n_adA * n_yes / n,
                            n_adB * n_yes / n],
                     'しなかった': [n_adA * n_not / n,
                              n_adB * n_not / n]},
                    index=['A', 'B'])
ad_ef

,した,しなかった
A,40.0,360.0
B,60.0,540.0


In [43]:
y = ((ad_cross - ad_ef)**2 / ad_ef).sum().sum()
y

3.75

In [44]:
rv = stats.chi2(1)
1 - rv.cdf(y)

0.052807511416113395

In [45]:
chi2, p, dof, ef = stats.chi2_contingency(ad_cross,
                                         correction=False)
chi2, p, dof

(3.75, 0.052807511416113395, 1)

In [46]:
ef

array([[ 40., 360.],
       [ 60., 540.]])